In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import terminalLogic
from terminalLogic import Algo
import client
from client import engine
import pandas as pd
import numpy as np
import datetime
import time


In [2]:
url = engine()
data = url.getData()
df = data
df
python olhc.py

SyntaxError: invalid syntax (<ipython-input-2-62d5067f76b0>, line 5)

In [ ]:
df

In [ ]:
df1 = Algo.fullstate(df, df)
df1.dropna(inplace=True)
df2 = df1
df2['time'] = df2['time'].dt.date
df2

In [ ]:
a = Algo.plot_positionT(df)
a

In [ ]:
df2

## Trend positions

In [ ]:
plt.figure(figsize=(30, 10))
fig = plt.figure(facecolor=(1, 1, 1))
y = df2.iloc[-500:]['time']
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(df2.iloc[-500:]['close'], label = 'BTC')
plt.plot(df2.iloc[-500:]['9-min'], label = '9-min')
plt.plot(df2.iloc[-500:]['21-min'], label = '21-min')
plt.plot(df2[-500:].loc[df2.entry == 2].index, df2[-500:]['9-min'][df2.entry == 2], "^",
        color = "g", markersize = 12, label= "Long")
plt.plot(df2[-500:].loc[df2.entry == -2].index, df2[-500:]['21-min'][df2.entry == -2], "v",
        color = "r", markersize = 12, label="Short")
plt.legend(loc=2);
plt.savefig('../web/assets/trendPositions.png')

# Trend Algo Performance

In [ ]:
fig = plt.figure(facecolor=(1, 1, 1))
y=df2.iloc[-500:]['time']
df2['trend_returns'] = df2.signal * df2.market_returns
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(np.exp(df2.iloc[-500:]['market_returns']).cumprod(),label = "Buy/Hold")
plt.plot(np.exp(df2.iloc[-500:]['trend_returns']).cumprod(),label = "Strat")
plt.legend()
plt.savefig('../web/assets/trendRets.png')

In [ ]:
trendRets = Algo.trend_gainz(df2)
trendRets

In [ ]:
fig = plt.figure(facecolor=(1, 1, 1))
y=df2.iloc[-500:]['time']
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(df2.iloc[-500:]['close'], label = 'BTC')
plt.plot(df2.iloc[-500:]['9-min'], label = '9-min')
plt.legend(loc=2);
plt.savefig('../web/assets/btc1m9ma.png')

# Range positions

In [ ]:
fig = plt.figure(facecolor=(1, 1, 1))
y=df2.iloc[-500:]['time']
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(df2.iloc[-500:]['position'].dropna())
plt.savefig('../web/assets/rangeStatus.png')


In [ ]:
p = Algo.plot_percentiles(df2.head(-500))
p

In [ ]:
""" Plots returns for the range algo df thus returns 'R' """
y=df2.iloc[-500:]['time'].dropna()
fig = plt.figure(facecolor=(1, 1, 1))
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(np.exp(df2.iloc[-500:]['market_returns'].dropna()).cumprod(), label='Buy/Hold')
plt.plot(np.exp(df2.iloc[-500:]['range_returns'].dropna()).cumprod(), label='Strategy')
plt.legend();
plt.savefig('../web/assets/rangeRets.png')

In [ ]:

y=df2.iloc[-500:]['time']
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(df2.iloc[-500:]['close'], label = 'BTC')
plt.plot(df2.iloc[-500:]['9-min'], label = '9-min')
plt.plot(df2[-500:].loc[df2.entryR == 2].index, df2[-500:]['9-min'][df2.entryR == 2], "^",
        color = "r", markersize = 12, label= "Short")
plt.plot(df2[-500:].loc[df2.entryR == -2].index, df2[-500:]['9-min'][df2.entryR == -2], "v",
        color = "g", markersize = 12, label="Long")
plt.legend(loc=2);
plt.savefig('../web/assets/dualPlot.png')


In [ ]:
rangeRet = Algo.market_returnsR(df2)
rangeRet

In [ ]:
rangeRets = Algo.range_gainz(df2)
rangeRets

In [ ]:
bestFit = df2[['time','close', 'market_returns','range_returns', 'trend_returns']]
bestFit.head(30)

* check val in each algo_return, determine greatest val, declear winner
* if declared winner, iterate forward len(X_minutes) in winner col and append new val to list
* check both val at winning location for greatest val, decleare winner
* iterate forward len(X_min) in winning col and append val to list
* repeat, i think..

In [ ]:
count=[]
selectVal = []
greatestVal = []
signal = ''
period = 3

for j, k in bestFit.iterrows():
    count.append(1)    
    if len(count) == period:
        selectVal.append(["time", k['market_returns'], "rangeRet", k['range_returns'], "trendRet", k['trend_returns']])
        if k['range_returns'] > k['trend_returns']:
            signal = -1
            greatestVal.append([ signal, k['range_returns']])
#             selectVal.append([k['range_returns'], signal])
#             np.insert(winnerVal, 1, greatestVal, axis = 1)
        elif k['range_returns'] < k['trend_returns']:
            signal = 1
            greatestVal.append([signal, k['trend_returns']])
#             selectVal.append([k['trend_returns'], signal])
        else:
            k['range_returns'] == k['trend_returns']
            signal = 0
            greatestVal.append([signal, k['trend_returns']])
#             selectVal.append([k['trend_returns'], signal])
        count = []

record=np.concatenate((selectVal,greatestVal), axis=1)
record

        
df3 = pd.DataFrame((record),
                     columns=['time', 'market_returns', 'range','rangeRet', 'trend', 'trendRet', 'winningAlgo', 'winnersRets'])   
df3.drop(['range', 'trend'], axis = 1, inplace=True)
record


In [ ]:
df3['addTrend'] = df3['trendRet'].shift(-1)
df3['addRange'] = df3['rangeRet'].shift(-1)
df3['result'] = np.where(df3['trendRet'] > df3['rangeRet'], df3['addTrend'], np.nan)
df3['result'] = np.where(df3['trendRet'] < df3['rangeRet'], df3['addRange'], df3['result'])
df3['result'] = np.where(df3['trendRet'] == df3['rangeRet'], df3['addRange'], df3['result'])
df3

In [ ]:
overview = pd.DataFrame(columns=['TotalTradesTaken','WinningTrades',
                                 'MaxWin', 'MinWin', 'LosingTrades',
                                 'MaxLoss', 'MinLoss', 'ZScore',
                                'SharpeRatio', 'Win/LossRatio',
                                '30DayPnL'])
overview

In [ ]:
df3.set_index('rangeRet')
df4 = df3.iloc[:-1 , :]
print(f' Dual Algo Returns over dataset 1minBTC: ', df3['result'].astype('float64').sum())
print(f'Market Buy/Hold Returns for same period: ', np.exp(bestFit.market_returns).astype('float64').cumprod().iloc[-1])
# when i try to sum it concatenates elements as strings, failed to convert to integer val
# also y output col have values that dont seem to be in range with the other values?

In [ ]:
plt.plot(np.exp(df3['market_returns'].dropna().astype('float64')).cumprod(), label='Buy/Hold')
plt.plot(np.exp(df3['result'].dropna().astype('float64')).cumprod(), label='Strategy')
plt.xticks(rotation=90)

In [ ]:
initial_capital = float(100000.0)

# Create df positions
positions = pd.DataFrame(index=df2.time.index).fillna(0.0)


# Buy 2 BTC
positions['BTC-PERP'] = 2*df['position']

# Initilize portfolio w value owned
portfolio = positions.multiply(df['close'], axis=0)

# Store diff in shares owned
pos_diff = positions.diff()

# Add 'holdings' to portfolio
portfolio['holdings'] = (positions.multiply(df['close'], axis=0)).sum(axis=1)

# Add 'cash' to portfolio
portfolio['cash'] = initial_capital - (pos_diff.multiply(df['close'], axis=0)).sum(axis=1).cumsum()

# Add 'total' to portfolio
portfolio['total'] = portfolio['cash'] + portfolio['holdings']

# Add 'returns' to portfolio
portfolio['returns'] = portfolio['total'].pct_change()

portfolio['time'] = df['time']
portfolio